<a href="https://colab.research.google.com/github/poet-developer/getPoemLineData_with-wikisource/blob/main/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C_%EB%8B%A4%EC%A4%91%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 위키문헌(Wikisource) XML파일 처리

In [1]:
pip install -q wikiextractor gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 922.3 kB/s eta 0:00:00


In [2]:
# 필요 패키지 load
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
# import gutenbergpy.textget
from pprint import pprint
import gdown
from collections import Counter
from itertools import chain

### 위키문헌 (wikisource)
https://dumps.wikimedia.org/kowikisource/20240901/  
참고문헌 : 서재현, 김병준, 김민우 and 박소정. (2021). 멀리서 읽는 “우리”― Word2Vec, N-gram을 이용한 근대 소설 텍스트 분석. 대동문화연구, 115, 349-386.

##### 위키문헌 전처리

In [3]:
# 위키문헌 xml 원본 파일 다운로드
# https://dumps.wikimedia.org/kowikisource/20240901/kowikisource-20240901-pages-articles.xml.bz2
!wget https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2

--2024-11-20 04:51:14--  https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151135574 (144M) [application/octet-stream]
Saving to: ‘kowikisource-20241101-pages-articles.xml.bz2’

kowikisource-202411 100%[===================>] 144.13M  3.43MB/s    in 41s     

2024-11-20 04:51:55 (3.49 MB/s) - ‘kowikisource-20241101-pages-articles.xml.bz2’ saved [151135574/151135574]



In [4]:
# Parsing (3분 가량 걸림)
!python -m wikiextractor.WikiExtractor kowikisource-20241101-pages-articles.xml.bz2

INFO: Preprocessing 'kowikisource-20241101-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Loaded 1242 templates in 67.3s
INFO: Starting page extraction from kowikisource-20241101-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Finished 1-process extraction of 37057 articles in 148.5s (249.5 art/s)


In [5]:
# 파싱한 파일을 하나의 xml로 합치기
!cat text/*/* > kowikisource.xml

In [6]:
# 맨위에 추가
!echo "$(echo -n -e '<?xml version="1.0" encoding="UTF-8"?>\n<wikisource>\n'; cat kowikisource.xml)" > kowikisource.xml

In [7]:
!head -n 20 kowikisource.xml # 첫 20줄만 확인

<?xml version="1.0" encoding="UTF-8"?>
<wikisource>
<doc id="1" url="https://ko.wikisource.org/wiki?curid=1" title="애국가 (대한민국)">
애국가 (대한민국)

한자 혼용
한글

</doc>
<doc id="2" url="https://ko.wikisource.org/wiki?curid=2" title="대한민국헌법 (한자혼용)">
대한민국헌법 (한자혼용)



</doc>
<doc id="3" url="https://ko.wikisource.org/wiki?curid=3" title="조선민주주의인민공화국 사회주의헌법">
조선민주주의인민공화국 사회주의헌법

서문.
조선민주주의인민공화국은 위대한 김일성동지와 김정일동지의 사상과 령도를 구현한 주체의 사회주의조국이다.


In [8]:
# 맨 아랫줄에 추가
!echo '</wikisource>' >> kowikisource.xml

In [9]:
!tail kowikisource.xml # 마지막줄 확인



</doc>
<doc id="93454" url="https://ko.wikisource.org/wiki?curid=93454" title="에티카 (스피노자)">
에티카 (스피노자)



</doc>
</wikisource>


##### 위키문헌 다루기

In [10]:
# xml 파일 dataframe로 읽기
df = pd.read_xml('./kowikisource.xml')

####작업자: 임이로
####poetdeveloper.iro@gmail.com
####bkksg.com

### 분리 행 추출 함수(getPoemLines)

In [58]:
def process_getPoemLines(data):
    # 불필요한 인자 제거
    filtered_list = [item for item in data if item not in [None, "", " ", [], {}, '<poem>', '</poem>']]

    # 제목 편집 (0번 인자 수정)
    pick_title = filtered_list[0].split('/')
    pick_title = pick_title[1] if len(pick_title) > 1 else filtered_list[0]
    filtered_list[0] = pick_title

    # 'title'은 첫 번째 인자, '본문'은 나머지 인자들
    title = filtered_list[0]
    body = filtered_list[1:]

    # DataFrame 생성: 본문 열이 먼저 오도록
    df = pd.DataFrame({'본문': body, '제목': [title] * len(body)})

    return df

### 시 전체 추출 함수(getPoemFull Lines)

In [59]:
def process_getPoemFullLines(data):
    # 불필요한 인자 제거
    cleaned_text = data.replace('<poem>', '').replace('</poem>', '').strip()
    # 텍스트를 줄 단위로 나누기
    lines = cleaned_text.strip().split('\n')
    title = lines[0]  # 첫 줄은 제목
    body = '\n'.join(lines[1:])  # 나머지는 본문

    # DataFrame 생성
    df = pd.DataFrame({'제목': [title], '본문': [body]})

    return df

### 검색 데이터를 가지고 특정 행마다 다중 추출(update_results_with_title)

In [13]:
def update_results_with_title(df, results_df, process_function, title_to_search, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")


    # 조건 반복문으로 'doc' 처리
    for index, row in filtered_text.iterrows():
        if 'doc' in row:  # 'doc' 열이 존재하는지 확인
            # 'doc' 값 추출
            text_value = row['doc']

            # 텍스트 처리
            processed_result = process_function(text_value.split('\n'))

            # 처리 결과를 기존 DataFrame에 추가
            results_df = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

            # # 새로운 열 추가 (고정된 값)
            results_df['저자'] = author
        else:
            print(f"Row {index}에 'doc' 값이 없습니다. 건너뜁니다.")
    return results_df

### 검색 데이터를 가지고 특정 시 전체 다중 추출(update results for FullText)

In [14]:
def update_FullTextResults_with_title(df, results_df, process_function, title_to_search, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")


    # 조건 반복문으로 'doc' 처리
    for index, row in filtered_text.iterrows():
        if 'doc' in row:  # 'doc' 열이 존재하는지 확인
            # 'doc' 값 추출
            text_value = row['doc']

            # 텍스트 처리
            processed_result = process_function(text_value)

            # 처리 결과를 기존 DataFrame에 추가
            results_df = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

            # # 새로운 열 추가 (고정된 값)
            results_df['저자'] = author
        else:
            print(f"Row {index}에 'doc' 값이 없습니다. 건너뜁니다.")
    return results_df

### 특정 데이터 추출 전 검색

In [132]:
# 검색 데이터 확인.
title = '강 건너간 노래'
author = '이육사'
check = df[df['title'].str.contains(title)]
check

,id,url,title,doc
634,3690,https://ko.wikisource.org/wiki?curid=3690,강 건너간 노래,\n강 건너간 노래\n\n<poem>\n섣달에도 보름ᄭᅦ 달발근밤\n압내江 ᄶᅢᆼᄶ...


In [127]:
#빈 프레임 지정. 열 지정.
text1 = pd.DataFrame(columns=['본문', '제목'])
text1

,본문,제목


### 데이터 추출 (행마다)

In [125]:
text1 = update_results_with_title(df, text1, process_getPoemLines, title, author)
text1.iloc[-1] # 마지막줄 확인

,304
본문,이 아름다운 詩集에 이 拙한 跋文을 부침이 또한 아름다운 인연이라고 불려지기를 가만...
제목,발문
저자,정지용


### 데이터 추출 (전체)

In [128]:
text1 = update_FullTextResults_with_title(df, text1, process_getPoemFullLines, title, author)
text1.iloc[-1] # 마지막줄 확인

,10
본문,\n천재 있는 詩人이 자기의 制作을 한번 지나가버린 길이오 넘어간 책장같이 여겨 그...
제목,정지용 시집/발문
저자,정지용


### 엑셀로 저장

In [129]:
# 엑셀로 저장하기
text1.to_excel('위키문헌_정지용_시집_전체행.xlsx')